In [1]:
# %pip install shap
# %pip install lime
#%pip install scikit-learn==1.3.2

# %pip install xgboost



In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Wczytanie danych
data = pd.read_csv('train.csv')

# Usunięcie brakujących wartości
data = data.dropna().reset_index(drop=True)

# One-Hot Encoding dla kolumny 'Profession'
profession_encoder = OneHotEncoder(sparse=False)
profession_encoded = profession_encoder.fit_transform(data[['Profession']])
profession_df = pd.DataFrame(profession_encoded, columns=profession_encoder.get_feature_names_out(['Profession']))
data = pd.concat([data.drop(['Profession'], axis=1), profession_df], axis=1)

# One-Hot Encoding dla kolumny 'Var_1'
var_1_encoder = OneHotEncoder(sparse=False)
var_1_encoded = var_1_encoder.fit_transform(data[['Var_1']])
var_1_df = pd.DataFrame(var_1_encoded, columns=var_1_encoder.get_feature_names_out(['Var_1']))
data = pd.concat([data.drop(['Var_1'], axis=1), var_1_df], axis=1)

# Label Encoding dla kolumn kategorycznych
label_encoder = LabelEncoder()
categorical_columns = ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score', 'Segmentation']
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

# Podział danych na zbiór treningowy i testowy
X = data.drop(['Segmentation', 'ID', 'Age'], axis=1)
y = data['Segmentation']

cols_X = X.columns
scaler = MinMaxScaler()
X= scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skalowanie danych


################################################################3
# Inicjalizacja i dopasowanie modelu Random Forest
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)

# Ocena modelu
y_pred = model_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność modelu Random Forest:", accuracy)




################################################################
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=4, random_state=42)
xgb_clf.fit(X_train, y_train)

# Przewidywanie klas dla danych testowych
y_pred = xgb_clf.predict(X_test)

# Obliczenie dokładności modelu
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność modelu XGBoost:", accuracy)



#########################################3

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Inicjalizacja klasyfikatora Gradient Boosting
gb_clf = GradientBoostingClassifier(random_state=42)

# Dopasowanie modelu do danych treningowych
gb_clf.fit(X_train, y_train)

# Przewidywanie klas dla danych testowych
y_pred_gb = gb_clf.predict(X_test)

# Obliczenie dokładności modelu
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Dokładność modelu Gradient Boosting:", accuracy_gb)



########################################

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Inicjalizacja klasyfikatora SVM
svm_clf = SVC(kernel='linear', decision_function_shape='ovr', random_state=42)

# Dopasowanie modelu do danych treningowych
svm_clf.fit(X_train, y_train)

# Przewidywanie klas dla danych testowych
y_pred_svm = svm_clf.predict(X_test)

# Obliczenie dokładności modelu
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Dokładność modelu SVM:", accuracy_svm)


######################################################
export = pd.DataFrame(X_test, columns=cols_X)
export['Seg'] = y
export.to_csv('xddd.csv')


/home/ryba/Documents/Code/snek/SoftwareRevolutionAI_BHL/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/ryba/Documents/Code/snek/SoftwareRevolutionAI_BHL/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Dokładność modelu Random Forest: 0.45911477869467365
Dokładność modelu XGBoost: 0.4973743435858965
Dokładność modelu Gradient Boosting: 0.5123780945236309
Dokładność modelu SVM: 0.4816204051012753


In [6]:

# POTEM, jesli w ogole
import numpy as np
import dalex as dx
result_shap = []
idx_X = np.random.randint(low=0, high=len(X_test), size=10)

for i in range(0,4):
    result_shap.append([])
    pf = lambda m, d: m.predict_proba(d)[:, i]
    exp = dx.Explainer(gb_clf, pd.DataFrame(X_test, columns=cols_X), y_test,  predict_function=pf)

    for j in range(len(idx_X)):
        result_shap[i].append(exp.predict_parts(X_test[idx_X[j]], N=10, type="shap").result)

    # Debug plots
    # ins_exp.plot()
    # exp.model_parts().plot()
    # ld.result
    al = exp.model_profile(type = 'accumulated')
    al.result['_label_'] = f'AL profiles{['a','b','c','d'][i]}'
    al.plot()
    exp.model_diagnostics().plot()
print(result_shap)

warnings.filterwarnings("default", category=UserWarning)
warnings.filterwarnings("default", category=FutureWarning)


SyntaxError: f-string: unmatched '[' (948363241.py, line 20)

In [ ]:
# %pip install statsmodels
# %pip install dalex

import dalex as dx
import numpy as np
import warnings


def calculate_shap(model, X, y, instance, cls_num, N=1000):
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)

    result = []
    for i in range(0, cls_num):
        
        pf = lambda m, d: m.predict_proba(d)[:, i]
        exp = dx.Explainer(model, X, y,  predict_function=pf)
        result.append(exp.predict_parts(instance, type="shap").result)
    warnings.filterwarnings("default", category=UserWarning)
    warnings.filterwarnings("default", category=FutureWarning)

    return result


# Sprawdzanie 
res = calculate_shap(gb_clf, pd.DataFrame(X_test, columns=cols_X), y_test, X_test[0], 4)
res[0]


Preparation of a new explainer is initiated

  -> data              : 1333 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1333 values
  -> model_class       : sklearn.ensemble._gb.GradientBoostingClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function calculate_shap.<locals>.<lambda> at 0x7fdcd8993c40> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0169, mean = 0.242, max = 0.729
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.729, mean = 1.34, max = 2.98
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 1333 rows 22 cols
  -> target variable   :

,variable,contribution,variable_name,variable_value,sign,label,B
0,Profession_Engineer = 0.0,-0.005527,Profession_Engineer,0.0,-1.0,GradientBoostingClassifier,1
1,Var_1_Cat_4 = 0.0,-0.004465,Var_1_Cat_4,0.0,-1.0,GradientBoostingClassifier,1
2,Profession_Artist = 0.0,0.015823,Profession_Artist,0.0,1.0,GradientBoostingClassifier,1
3,Var_1_Cat_3 = 1.0,0.015538,Var_1_Cat_3,1.0,1.0,GradientBoostingClassifier,1
4,Profession_Lawyer = 0.0,-0.002081,Profession_Lawyer,0.0,-1.0,GradientBoostingClassifier,1
...,...,...,...,...,...,...,...
17,Spending_Score = 1.0,-0.000586,Spending_Score,1.0,-1.0,GradientBoostingClassifier,0
18,Profession_Lawyer = 0.0,-0.000402,Profession_Lawyer,0.0,-1.0,GradientBoostingClassifier,0
19,Var_1_Cat_1 = 0.0,0.000361,Var_1_Cat_1,0.0,1.0,GradientBoostingClassifier,0
20,Var_1_Cat_5 = 0.0,-0.000155,Var_1_Cat_5,0.0,-1.0,GradientBoostingClassifier,0


In [ ]:
print(type(X))


<class 'numpy.ndarray'>
